# Chatbot com RAG

RAG (Retrieval Augmented Generation) é a estratégia aplicada a LLMs para gerar a resposta utilizando informações recuperadas de fontes externas a partir da consulta realizada. Neste caso, o prompt enviado ao LLM contém as informações recuperadas e a pergunta inicial.

RAG é útil para combater o problema da alucinação em LLMs.

## Configuração

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
import logging

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)


## Componentes

Define o LLM a ser usado para responder perguntas.

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2", 
    temperature=0.0, 
    max_tokens=2000,
)

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="llama3-8b-8192", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

Define o modelo de embeddings usado para vetorizar os documentos.

Veja mais em: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [ ]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="granite-embedding:278m")
vetorial_dbname = "granite_embedding_278m"

Mais informações e outras opções de modelos de embeddings podem ser encontradas em:

- https://python.langchain.com/docs/concepts/embedding_models/
- https://python.langchain.com/docs/integrations/text_embedding/
- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
- https://arxiv.org/abs/2407.19527
- https://huggingface.co/models?pipeline_tag=sentence-similarity&language=pt&sort=trending


In [ ]:
# from langchain_ollama import OllamaEmbeddings

# embedding_model = OllamaEmbeddings(model="nomic-embed-text")
# vetorial_dbname = "nomic_embed_text"

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings

# embedding_model = HuggingFaceEmbeddings(model_name="PORTULAN/serafim-100m-portuguese-pt-sentence-encoder-ir")
# vetorial_dbname = "serafim_100m_ir"

Define o banco de dados vetorial que armazena os documentos e permite busca.

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="chatbot",
    embedding_function=embedding_model,
    persist_directory=f"./chroma_db/{vetorial_dbname}",
)

## [*] Indexa os documentos

Indexa os documentos na base vetorial, usando a representação de blocos de texto.

Lê os documentos.

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

loader = WikipediaLoader(query="Campinas", load_max_docs=1, lang="pt", doc_content_chars_max=-1)

docs = loader.load()

print("Documentos:", len(docs))
print("Tamanho (chars):", len(docs[0].page_content))
print("="*80, docs[0].page_content[:1000], "[...]", sep="\n")

Separa os documentos em blocos.

Quando realizamos uma busca, o sistema recupera os blocos relacionados ao termo de busca.

Isso permite respeitar o tamanho do contexto do LLM e também diminui o tempo de resposta.

In [ ]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

print("Blocos:", len(all_splits))

Adiciona os documentos na Vector Store.

(descomente o código caso queira executar)

In [ ]:
# import tqdm

# print("vetorial_dbname:", vetorial_dbname)

# for doc in tqdm.tqdm(all_splits, desc="Adicionando documentos à Vector Store"):
#     vector_store.add_documents(documents=[doc])

## Recupera os documentos

Define o *retriever*, que recupera documentos (na verdade, blocos) a partir da representação da consulta realizada.

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

Recupera os documentos relacionados à consulta

In [ ]:
from typing import List
from langchain_core.documents import Document

def print_docs(docs: List[Document]):
    print("### Documentos:", len(docs))
    print("="*80)
    for doc in docs:
        print(doc.page_content[:1000], "="*80, sep="\n")

In [ ]:
print("Usando o retriever:", vetorial_dbname)

# question = "Qual a população de Campinas?"
# question = "Quais os terminais de ônibus em Campinas?"
# question = "Qual o papel da agricultura na economia de Campinas?"
# question = "Quais são as universidades existentes em Campinas?"
# question = "Quais os centros de pesquisa de Campinas?"
question = "Quando a cidade foi fundada e qual era a base da economia?"

retrieved_docs = retriever.invoke(question)
print_docs(retrieved_docs)

## Aplica ReRanking

Adicionando ReRanker.

- https://python.langchain.com/docs/integrations/retrievers/flashrank-reranker/
- https://python.langchain.com/docs/integrations/document_transformers/cross_encoder_reranker/ 
- https://github.com/PrithivirajDamodaran/FlashRank
- https://www.sbert.net/examples/cross_encoder/applications/README.html
- https://huggingface.co/Alibaba-NLP/gte-multilingual-reranker-base
- https://huggingface.co/cross-encoder 


ReRanking rápido:

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.flashrank_rerank import FlashrankRerank

# Mantém somente os 3 melhores
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
retrieved_docs = compression_retriever.invoke(question)
print_docs(retrieved_docs)

Outro tipo de modelo para ReRanking (mais lento):

In [ ]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import CrossEncoderReranker
# from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# model = HuggingFaceCrossEncoder(
#     model_name="Alibaba-NLP/gte-multilingual-reranker-base",
#     model_kwargs={"trust_remote_code": True},
# )
# compressor = CrossEncoderReranker(model=model, top_n=3)
# base_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=base_retriever
# )

# retrieved_docs = compression_retriever.invoke(question)
# print_docs(retrieved_docs)

## Realiza a consulta

Prepara o prompt

In [ ]:
from typing import List
from langchain_core.documents import Document


prompt_template = """Utilizando apenas as informações disponíveis, responda a pergunta.
Se a pergunta não puder ser respondida a partir dessas informações, explique que não é possível responder a pergunta.

### Informações disponíveis
{context}

### Pergunta
{question}

### Resposta
"""


def build_prompt(question: str, retrieved_docs: List[Document]):
    # Monta o texto com todos os documentos recuperados
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # Monta o prompt usando o template
    prompt = prompt_template.format(question=question, context=docs_content)
    return prompt

Chama o LLM usando o prompt construído

In [ ]:
prompt = build_prompt(question, retrieved_docs)
response = llm.invoke(prompt)
print(prompt)
print("="*80)
print(response.content)

Escreve a resposta token por token.

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end="", flush=True)

## Aplicação simples usando Gradio

In [ ]:
def chatbot(question: str, history=None):
    # O argumento history é passado pelo Gradio com o ChatInterface
    retrieved_docs = compression_retriever.invoke(question)
    prompt = build_prompt(question, retrieved_docs)
    response = llm.invoke(prompt).content
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return response, docs_content


In [ ]:
result = chatbot("Qual a população de Campinas?")
result[0]

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Chatbot com RAG")
    with gr.Accordion("Outras informações", open=False):
        retrieved_docs = gr.TextArea(label="Documentos Recuperados")
    gr.ChatInterface(
        fn=chatbot,
        additional_outputs=[retrieved_docs],
        type="messages",
    )

demo.launch(inline=False)

Fecha a aplicação

In [1]:
try:
    demo.close()
except NameError:
    pass